In [87]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Hide/ View Code</button>''', raw=True)

Hide/ View Code

In [83]:
%pylab inline

import pandas as pd
import pickle as pk
from scipy import sparse as sp

from fastparquet import ParquetFile

tweet_df = pd.read_csv("outputCSV/part-00001-5b1aa25d-22c8-413f-aa11-e4b5d2aae996-c000.csv", error_bad_lines=False, warn_bad_lines=False)

Populating the interactive namespace from numpy and matplotlib


In [54]:
docs = array(tweet_df['tweet'])

In [55]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [56]:
docs = docs_preprocessor(docs)

In [57]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

/anaconda/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [59]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [62]:
from gensim.models import LdaModel

In [69]:
num_topics = 7
chunksize = 500 
passes = 20 
iterations = 400
eval_every = 1  

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 12min 30s, sys: 2.41 s, total: 12min 32s
Wall time: 12min 37s


In [70]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [71]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      27.140305        1       1 -0.044159 -0.150252
6      21.769662        1       2  0.054613  0.349693
3      15.010689        1       3 -0.240271  0.056875
0      10.397650        1       4 -0.236794  0.055914
4       9.908143        1       5 -0.051581 -0.250521
1       9.481961        1       6  0.277474  0.084489
5       6.291589        1       7  0.240718 -0.146199, topic_info=     Category          Freq              Term         Total  loglift  logprob
term                                                                         
41    Default  17013.000000              this  17013.000000  30.0000  30.0000
215   Default  13378.000000              that  13378.000000  29.0000  29.0000
59    Default   7644.000000              with   7644.000000  28.0000  28.0000
135   Default   6471.000000              have   6471.000000  27.0000  27.0000
1     Default   7178.000000              just   7178.000000  26.0000  26.0000
78    Default   5705.000000              what   5705.000000  25.0000  25.0000
27    Default   3436.000000  hiring_careerarc   3436.000000  24.0000  24.0000
2     Default   5795.000000              like   5795.000000  23.0000  23.0000
26    Default   2968.000000              here   2968.000000  22.0000  22.0000
89    Default   5251.000000              they   5251.000000  21.0000  21.0000
218   Default   2795.000000             great   2795.000000  20.0000  20.0000
115   Default   5020.000000              your   5020.000000  19.0000  19.0000
21    Default   3346.000000            hiring   3346.000000  18.0000  18.0000
203   Default   4672.000000              love   4672.000000  17.0000  17.0000
57    Default   3993.000000              time   3993.000000  16.0000  16.0000
124   Default   2194.000000         want_work   2194.000000  15.0000  15.0000
7     Default   3563.000000              want   3563.000000  14.0000  14.0000
42    Default   2867.000000              work   2867.000000  13.0000  13.0000
156   Default   3435.000000              from   3435.000000  12.0000  12.0000
273   Default   2182.000000  recommend_anyone   2182.000000  11.0000  11.0000
704   Default   2221.000000              shit   2221.000000  10.0000  10.0000
25    Default   2708.000000         careerarc   2708.000000   9.0000   9.0000
257   Default   2141.000000             never   2141.000000   8.0000   8.0000
179   Default   3696.000000              when   3696.000000   7.0000   7.0000
695   Default   3575.000000              good   3575.000000   6.0000   6.0000
83    Default   2047.000000              more   2047.000000   5.0000   5.0000
75    Default   1982.000000           tonight   1982.000000   4.0000   4.0000
102   Default   1928.000000              look   1928.000000   3.0000   3.0000
427   Default   3392.000000              know   3392.000000   2.0000   2.0000
740   Default   2895.000000              back   2895.000000   1.0000   1.0000
...       ...           ...               ...           ...      ...      ...
118    Topic7    854.636064               job    854.824865   2.7657  -4.1018
121    Topic7    823.750607      click_detail    823.941156   2.7657  -4.1386
16     Topic7    766.305652             apply    766.494414   2.7657  -4.2109
119    Topic7    711.270214     job_careerarc    711.458876   2.7657  -4.2854
471    Topic7    696.947684        job_hiring    697.134826   2.7657  -4.3058
559    Topic7    667.055498          watching    667.243502   2.7657  -4.3496
433    Topic7    646.284877              view    646.471790   2.7657  -4.3812
558    Topic7    514.871879      this_morning    515.059607   2.7656  -4.6086
431    Topic7    470.368958      opening_http    470.554141   2.7656  -4.6990
788    Topic7    468.402814           awesome    468.589299   2.7656  -4.7032
5659   Topic7    396.065486       benny_snell    396.239414   2.7655  -4.8709
1214   Topic7    395.134474            